In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import os

In [2]:
# Load dataset
hebrew_folder_path = r"Data\Hebrew"
english_folder_path = r"Data\English"
dataset_path = r"Data\Modified"

In [3]:
def combine_csvs(path):
    df = pd.DataFrame({})
    files = os.listdir(path)
    print(files)
    for file in files:
        file_path = os.path.join(path, file)
        tmp = pd.read_csv(file_path)
        df = pd.concat([df, tmp], ignore_index=True)
    return df

df = combine_csvs(dataset_path)
df

['Aquaman.2018.csv', 'Ice Age Continental Drift.csv', 'Mission.Impossible.Dead.Reckoning.Part.One-modified.csv']


,Hebrew,English,Safe,Hebrew_st,Hebrew_e,English_st,English_e
0,", ג'ול ורנס כתב פעם","Jules Verne once wrote,",True,"00:01:05,990","00:01:07,920","00:01:06,109","00:01:07,444"
1,",שים שתי ספינות בים הפתוח""","""Put two ships in the open sea",True,"00:01:07,920","00:01:10,420","00:01:08,027","00:01:10,989"
2,",ללא רוח או שפל"", .""מתישהו, הן ייפגשו","without wind or tide,, they will come together.""",True,"00:01:10,420","00:01:13,760","00:01:11,156","00:01:13,575"
3,".ככה הוריי הכירו, .כמו שתי ספינות שנועדו להיפגש","That's how my parents met., Like two ships des...",True,"00:01:35,850","00:01:40,520","00:01:36,181","00:01:40,268"
4,".זה בסדר, זה בסדר",It's okay.,True,"00:01:54,400","00:01:55,600","00:01:54,282","00:01:55,283"
...,...,...,...,...,...,...,...
2430,.והעולם ישלם את המחיר האולטימטיבי,and the world will pay the ultimate price.,True,"02:34:42,961","02:34:45,714","02:34:42,983","02:34:45,444"
2431,.הקרבתו תהיה לשווא,their sacrifice will have been in vain.,True,"02:34:50,594","02:34:53,555","02:34:50,616","02:34:53,035"
2432,.אז תזדרז,So hurry.,True,"02:34:54,264","02:34:55,724","02:34:54,286","02:34:55,454"
2433,.אין הרבה זמן,There isn't much time.,True,"02:34:56,350","02:34:58,268","02:34:56,371","02:34:57,915"
